In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import geometry, ops
import folium

In [2]:
# Change the path for your own shapefile

fp = '/Users/santiagotoso/GoogleDrive/Master/Python/Orebro/SHP_c737300_2019-08-19-10-42-51/directions.shp'
gdf = gpd.read_file(fp)
gdf.head()

,full_name,dir_id,dir_name,pattern,line_id,line_name,map_id,map_name,url,mi,hours_year,cost_year,vehicle_ty,vehicle_co,minheadway,maxheadway,minmph,maxmph,geometry
0,Shapefile 8 - inbound,486834454,inbound,A,2060be8,Shapefile 8,c737300,Copy of Greencats,https://platform.remix.com/map/c737300/line/20...,0.679157,0.0,0.0,None,None,15,30,14.975016,14.975016,"LINESTRING (-84.019532 39.807424, -84.019532 3..."
1,Shapefile 7 - inbound,486834452,inbound,A,3318a6a,Shapefile 7,c737300,Copy of Greencats,https://platform.remix.com/map/c737300/line/33...,0.646846,0.0,0.0,None,None,15,30,14.975016,14.975016,"LINESTRING (-84.022173 39.821617, -84.022173 3..."
2,Shapefile 0 - inbound,486834438,inbound,A,4b39790,Shapefile 0,c737300,Copy of Greencats,https://platform.remix.com/map/c737300/line/4b...,1.094854,0.0,0.0,None,None,15,30,14.975016,14.975016,"LINESTRING (-84.023577 39.812328, -84.023577 3..."
3,Shapefile 2 - inbound,486834442,inbound,A,634ae9b,Shapefile 2,c737300,Copy of Greencats,https://platform.remix.com/map/c737300/line/63...,1.226584,0.0,0.0,None,None,15,30,14.975016,14.975016,"LINESTRING (-84.02060400000001 39.797634, -84...."
4,Shapefile 9 - inbound,486834456,inbound,A,8112a22,Shapefile 9,c737300,Copy of Greencats,https://platform.remix.com/map/c737300/line/81...,0.217479,0.0,0.0,None,None,15,30,14.975016,14.975016,"LINESTRING (-84.018092 39.821338, -84.018092 3..."


In [9]:
# Will create merge the rows of the shapefile into one feature

# Create an empty GeoDataFrame
shapefile = gpd.GeoDataFrame()

# Make sure the projection is right
gdf = gdf.to_crs({'init': 'epsg:4326'})
                
# Combine them into a multi-linestring
multi_line = geometry.MultiLineString(list(gdf.geometry))
                
# you can now merge the lines into one linestring (only in case the align)
merged_line = ops.linemerge(multi_line)
                
# create the geoseries
line_series = gpd.GeoSeries({'geometry': merged_line})
                
# append this shape to the shapefile
shapefile = shapefile.append(gpd.GeoDataFrame(data=line_series).T)

In [12]:
m = folium.Map(location = [39.81192,-84.0325], tiles='cartodbpositron', zoom_start=13)

folium.GeoJson(shapefile.__geo_interface__).add_to(m)

m

In [20]:
prj = 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]]'

shapefile.to_file(driver = 'ESRI Shapefile',
                filename = '/Users/santiagotoso/GoogleDrive/Master/Python/Orebro/SHP_c737300_2019-08-19-10-42-51/Blue Line.shp',
                encoding='utf-8',
                crs_wkt=prj)